In [ ]:
#from sys import path
#path.append('./funciones')

Importo el dataset de manera general

In [2]:
import pandas as pd
import numpy as np

data_location = './datasets/properatti.csv'

raw_data = pd.read_csv(data_location)

In [3]:
data = raw_data
data.index

RangeIndex(start=0, stop=121220, step=1)

Dataset

#Completar con nombre de campo y tipo de dato

Porcentaje de faltantes de datos

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [5]:
data.sample(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
11436,11436,sell,house,Escobar,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|,Argentina,Bs.As. G.B.A. Zona Norte,3434130.0,"-34.3451368808,-58.7823486328",-34.345137,...,160.0,257.985258,1312.500000,NaN,3.0,NaN,http://www.properati.com.ar/16bkx_venta_casa_e...,BARRIO CERRADO EL CANTON PUERTO CASA EN LOTE I...,CASA en VENTA en EL CANTON BARRIO CERRADO,https://thumbs4.properati.com/7/RYhRr6BiqfrnIr...
97094,97094,sell,house,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10000.0,http://www.properati.com.ar/1bjkn_venta_casa_p...,casa distribuida en dos plantas.PB: hall corre...,Casa en Tortugas C.C,https://thumbs4.properati.com/5/MlK7USRmo2Dpmi...
36267,36267,sell,apartment,San Vicente,|Argentina|Bs.As. G.B.A. Zona Sur|San Vicente|,Argentina,Bs.As. G.B.A. Zona Sur,3428056.0,NaN,NaN,...,83.0,3360.465116,3481.927711,4.0,NaN,NaN,http://www.properati.com.ar/18ggo_venta_depart...,Buen departamento en su estado original.Excele...,Departamento 86m² con Cochera en Del Libertado...,https://thumbs4.properati.com/9/ZCo6dzk2Sfx9rq...
6762,6762,sell,house,Escobar,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|,Argentina,Bs.As. G.B.A. Zona Norte,3434130.0,NaN,NaN,...,250.0,980.000000,980.000000,NaN,NaN,NaN,http://www.properati.com.ar/15vk9_venta_casa_e...,JPL. Casa en el barrio San Sebastian con bajad...,JPL. Casa en San Sebastian,https://thumbs4.properati.com/2/1LQgX0HlkYKSW5...
7988,7988,sell,apartment,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,"-34.6058091,-58.4534805",-34.605809,...,22.0,1986.111111,3250.000000,NaN,1.0,NaN,http://www.properati.com.ar/15zob_venta_depart...,"Emprendimiento ícono en la zona, Caballito Nor...",Departamento - Caballito Norte,https://thumbs4.properati.com/1/YN9StXiLKv20MH...
45054,45054,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,...,295.0,951.923077,1677.966102,NaN,NaN,NaN,http://www.properati.com.ar/191te_venta_casa_m...,Muy buena casa de excelente calidadPB: Hall de...,"Casa 520m² con 2 Plantas en Aramburu, Tte. Gra...",https://thumbs4.properati.com/8/PThgcDa1_eK2Hc...
20552,20552,sell,house,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.7545718,-58.4117264",-34.754572,...,250.0,1000.000000,1400.000000,NaN,5.0,NaN,http://www.properati.com.ar/17czh_venta_casa_l...,"Venta de Casa 5 AMBIENTES en LOMAS DE ZAMORA, ...",CASA EN VENTA,https://thumbs4.properati.com/2/EiQ7o06OnWhOHz...
71747,71747,sell,house,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,NaN,NaN,...,136.0,NaN,933.823529,NaN,NaN,NaN,http://www.properati.com.ar/1aldc_venta_casa_l...,None- Estado del inmueble: MUY BUENO- Tiene ga...,CASA DE 4 AMBIENTES EN VENTA,https://thumbs4.properati.com/5/ISHDn00YOuHP4-...
44283,44283,sell,house,Benavidez,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Bena...,Argentina,Bs.As. G.B.A. Zona Norte,3436048.0,NaN,NaN,...,119.0,NaN,3109.243697,NaN,NaN,NaN,http://www.properati.com.ar/1905p_venta_casa_b...,Excelente casa a la laguna muy original con di...,"Casa con Plantas, Tigre, Benavidez, por U$S 37...",https://thumbs4.properati.com/9/84_vE7h3EYN7Gb...
62459,62459,sell,apartment,Olivos,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430310.0,"-34.503342,-58.481441",-34.503342,...,140.0,2697.934066,3507.314286,NaN,4.0,NaN,http://www.properati.com.ar/1a73m_venta_depart...,Domus Libertador se encuentra en el histórico ...,"Cuatro Ambientes, Unidad (02,03,04)",https://thumbs4.properati.com/9/w_Wx7UspTbLL5w...


In [6]:
from funciones import *

In [7]:
missing_zero_values_table(data)

Your selected dataframe has 26 columns and 121220 Rows.
There are 18 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
floor,0,113321,93.48,113321,93.48,float64
expenses,0,106958,88.23,106958,88.23,float64
rooms,0,73830,60.91,73830,60.91,float64
price_usd_per_m2,0,52603,43.39,52603,43.39,float64
lat-lon,0,51550,42.53,51550,42.53,object
lat,0,51550,42.53,51550,42.53,float64
lon,0,51550,42.53,51550,42.53,float64
surface_total_in_m2,383,39328,32.44,39711,32.76,float64
price_per_m2,0,33562,27.69,33562,27.69,float64
currency,0,20411,16.84,20411,16.84,object


Nos deshacemos de la columna de indice ya que tienen los mismos valores.

In [8]:
#data.reset_index(drop=True, inplace=True)
#data.index

### Análisis de la variable _operation_

In [9]:
data['operation'].value_counts()

sell    121220
Name: operation, dtype: int64

Todas las propiedades son para la venta, lo cual representa una categoría única.

In [10]:
data['operation_category'] = data['operation'].astype('category')
data['operation_category'].dtype

CategoricalDtype(categories=['sell'], ordered=False)

### Análisis de la variable _property_type_

In [11]:
data['property_type'].value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

Observamos que tiene tiene cuatro posibles opciones por lo cual la cambiamos a categórica.

In [12]:
data['property_type_category'] = data['property_type'].astype('category')
data['property_type_category'].dtype

CategoricalDtype(categories=['PH', 'apartment', 'house', 'store'], ordered=False)

### Análisis de la variable _place_name_

In [13]:
data['place_name'].value_counts()
#data[['place_name','place_with_parent_names','country_name','state_name']].sample(10)

Córdoba                     9254
Rosario                     8504
Mar del Plata               6534
Tigre                       3324
Nordelta                    3315
                            ... 
Playa Serena                   1
La Arboleda de Maschwitz       1
El Viejo Vivero                1
Las Chacras                    1
Trenel                         1
Name: place_name, Length: 1060, dtype: int64

Haciendo un análisis de las 23 observaciones que no tienen 'place_name', observamos que las mismas pertenecen al barrio de Rincón de Mimberg, y hay observaciones que tienen las mismas características para descripciones similares. Sumado a que geográficamente las descripciones apuntan a que se encuentran en barrios cerrados ó courtries en dicha ubicación.

In [14]:
data_place_name_null = data.loc[data['place_name'].isnull()]
data_place_name_null.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category
6489,6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,...,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...,sell,house
10201,10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,...,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...,sell,house
11451,11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,...,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...,sell,house
14839,14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,...,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...,sell,house
18622,18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,...,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...,sell,house


In [15]:
data[data['place_name'].str.contains('Rincón de Milberg',na=False)].head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category
573,573,sell,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4084868477,-58.5920269766",-34.408487,...,3021.978022,NaN,3.0,NaN,http://www.properati.com.ar/15fcg_venta_depart...,"Excelente departamento a estrenar, edificio re...",Muy lindo depto de 3 amb con vista al rio en T...,https://thumbs4.properati.com/8/o_eatpL-3fJytW...,sell,apartment
590,590,sell,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4308974,-58.6882264",-34.430897,...,2426.470588,NaN,3.0,NaN,http://www.properati.com.ar/15fdm_venta_depart...,A solo 4 cuadras del acceso Ramal Tigre!!! COM...,"VENTA EXCELENTE departamento 3 ambientes , C...",https://thumbs4.properati.com/4/LiG_r0MIpHKCwk...,sell,apartment
630,630,sell,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4228922077,-58.5884990609",-34.422892,...,2580.645161,NaN,2.0,NaN,http://www.properati.com.ar/15ffq_venta_depart...,Departamento dos ambientes. Living/comedor con...,"Departamento 2 ambientes en planta baja,Tigre,...",https://thumbs4.properati.com/4/WrNT-tsDQvG9Kd...,sell,apartment
1079,1079,sell,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,2187.500000,12.0,NaN,NaN,http://www.properati.com.ar/15h5q_venta_depart...,Muy lindo departamento de 3 ambientes - LIving...,Departamento 70m² con Plantas en Montevideo 15...,https://thumbs4.properati.com/7/b5LRdhZibskJ_-...,sell,apartment
4919,4919,sell,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.3951514415,-58.6490096963",-34.395151,...,2884.615385,NaN,3.0,NaN,http://www.properati.com.ar/15ogo_venta_depart...,"CGI, Impecable 4 ambientes vista al rio,y bahi...","CGI, Vista Bahia, 4 ambientes vista al rio jar...",https://thumbs4.properati.com/5/69YPd-6YXgcFrw...,sell,apartment


Por este motivo imputamos a los datos faltantes con Rincón de Milberg, ajustando al mismo tiempo el campo place_with_parent_names para que sea consistente con la información del dataset.

In [16]:
data['place_name_check'] = data['place_name']
data['place_with_parent_names_check'] = data['place_with_parent_names']

data.loc[data['place_name'].isnull(),['place_name_check','place_with_parent_names_check']] = \
    ['Rincón de Milberg','|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rincón de Milberg|']
data_place_name_null

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category
6489,6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,...,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...,sell,house
10201,10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,...,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...,sell,house
11451,11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,...,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...,sell,house
14839,14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,...,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...,sell,house
18622,18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,...,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...,sell,house
21922,21922,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.399704,-58.638405",-34.399704,...,2000.000000,NaN,5.0,NaN,http://www.properati.com.ar/17gy5_venta_casa_s...,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,https://thumbs4.properati.com/8/oO7kmFPYNm8DYr...,sell,house
23664,23664,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4005161998,-58.6417999185",-34.400516,...,1632.653061,NaN,NaN,NaN,http://www.properati.com.ar/17mb5_venta_casa_t...,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,https://thumbs4.properati.com/2/LQCwH6txpFqetn...,sell,house
24722,24722,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007292,-58.6377783",-34.400729,...,1962.962963,NaN,NaN,NaN,http://www.properati.com.ar/17oz1_venta_casa_t...,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/0/Nhx4NejZaQACap...,sell,house
38856,38856,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.425087,-58.5796585",-34.425087,...,2454.545455,NaN,NaN,NaN,http://www.properati.com.ar/18mcb_venta_casa_t...,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",https://thumbs4.properati.com/2/Izq8d5o8ti3kJy...,sell,house
45970,45970,sell,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.396074,-58.63738",-34.396074,...,3518.518519,NaN,1.0,NaN,http://www.properati.com.ar/1958f_venta_depart...,"EDIFICIO ZERENA – EL PALMAR, NORDELTA. Torre 2...",Excelente monoambiente. Barrio El Palmar. Con...,https://thumb

Comprobamos que no haya faltantes en las columnas que incluimos extra

In [17]:
missing_zero_values_table(data)

Your selected dataframe has 30 columns and 121220 Rows.
There are 18 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
floor,0,113321,93.48,113321,93.48,float64
expenses,0,106958,88.23,106958,88.23,float64
rooms,0,73830,60.91,73830,60.91,float64
price_usd_per_m2,0,52603,43.39,52603,43.39,float64
lat-lon,0,51550,42.53,51550,42.53,object
lat,0,51550,42.53,51550,42.53,float64
lon,0,51550,42.53,51550,42.53,float64
surface_total_in_m2,383,39328,32.44,39711,32.76,float64
price_per_m2,0,33562,27.69,33562,27.69,float64
currency,0,20411,16.84,20411,16.84,object


### Análisis de la variable _country_name_

In [18]:
data['country_name'].value_counts()

Argentina    121220
Name: country_name, dtype: int64

No es necesario realizar ningún tipo de ajuste en esta variable. Vemos que todas las propiedades son en Argentina.

### Análisis de la variable _state_name_

In [19]:
data['state_name'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

No es necesario realizar ningún tipo de ajuste en esta variable. Observamos que la provinicia de Buenos Aires ha sido dividida en regiones geográficas, a diferencia del resto de las provincias.

### Análisis de las variables de Geoposicionamiento

Corroboramos que todas las observaciones que tienen datos en lat-lon tienen tambien en lat y lon

In [20]:
mask_lat_notnull = data['lat'].notnull()
mask_lon_notnull = data['lat'].notnull()
mask_lat_lon_notnull = data['lat-lon'].notnull()

mask_comparison_lat_lon = mask_lat_notnull & mask_lon_notnull & mask_lat_lon_notnull

print(f"Cantidad de observaciones con datos de Latitud y Longitud: {mask_lat_lon_notnull.sum()}")
print(f"Cantidad de observaciones en donde si hay datos de Latitud y Longitud también hay datos en las columnas separadas: {mask_comparison_lat_lon.sum()}")


data.loc[mask_comparison_lat_lon,['lat-lon','lat','lon']]

Cantidad de observaciones con datos de Latitud y Longitud: 69670
Cantidad de observaciones en donde si hay datos de Latitud y Longitud también hay datos en las columnas separadas: 69670


,lat-lon,lat,lon
0,"-34.6618237,-58.5088387",-34.661824,-58.508839
1,"-34.9038831,-57.9643295",-34.903883,-57.964330
2,"-34.6522615,-58.5229825",-34.652262,-58.522982
3,"-34.6477969,-58.5164244",-34.647797,-58.516424
4,"-38.0026256,-57.5494468",-38.002626,-57.549447
...,...,...,...
121153,"-34.5992135511,-58.4025191971",-34.599214,-58.402519
121158,"-34.587425443,-58.3973722534",-34.587425,-58.397372
121160,"-34.563685346,-58.4426825896",-34.563685,-58.442683
121209,"-34.563685346,-58.4426825896",-34.563685,-58.442683


Corroboramos que los valores sean los mismos luego de haber hecho la división en columnas

In [21]:
import re

pattern_lat_lon = '(?P<latitude>.+),(?P<longitude>.+)'
pattern_lat_lon_regex = re.compile(pattern_lat_lon)

resultados_lat_lon = data['lat-lon'].apply(lambda x: x if x is np.NaN else pattern_lat_lon_regex.match(x))

lat_match = resultados_lat_lon.apply(lambda x: x if x is np.NaN else x.group('latitude'))
lon_match = resultados_lat_lon.apply(lambda x: x if x is np.NaN else x.group('longitude'))


lat_match_numeric = lat_match.astype(float)
lon_match_numeric = lon_match.astype(float)


lat_check = data['lat'] - lat_match_numeric < .000001
lon_check = data['lon'] - lon_match_numeric < .000001


data_lat_lon_check = lat_check & lon_check

print(f"Cantidad de valores donde coinciden los valores de las columnas: {data_lat_lon_check.sum()}")


Cantidad de valores donde coinciden los valores de las columnas: 69670


Vamos a revisar las 

In [22]:
data['currency'].value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

In [39]:
data['currency'].notnull().sum()

mask_testing = (data['currency'].isnull()) \
        & ((data['price'].notnull()) \
        | (data['price_aprox_local_currency'].notnull()) \
        | (data['price_aprox_usd'].notnull()) \
        | (data['price_usd_per_m2'].notnull()) \
        | (data['price_per_m2'].notnull()))
mask_testing.sum()
data[mask_testing]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category,place_name_check,place_with_parent_names_check
9761,9761,sell,house,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,"4.5458426,-75.67893099",4.545843,...,3.0,NaN,http://www.properati.com.ar/165i3_venta_casa_c...,Casa en Venta de 2 dorm. en Armenia,SE VENDE CASA EN EL BARRIO EL LIMONAR,https://thumbs4.properati.com/2/2huBKPZv0RyWda...,sell,house,Córdoba,|Argentina|Córdoba|


Para aquellas observaciones para las cuales únicamente tenemos el geoname_id le obtenemos la latitud y longitud correspondiente

NO por ahora.

In [23]:
data.head(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category,place_name_check,place_with_parent_names_check
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,...,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|
9,9,sell,house,Rosario,|Argentina

In [24]:
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [25]:
data.loc[data['state_name']=='Capital Federal']['place_name'].unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Capital Federal',
       'Colegiales', 'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta',
       'Congreso', 'Villa Crespo', 'Chacarita', 'Constitución',
       'Villa Urquiza', 'Palermo Hollywood', 'Saavedra', 'Monserrat',
       'Pompeya', 'Parque Chas', 'Paternal', 'Agronomía',
       'Villa Pueyrredón', 'Coghlan', 'Parque Centenario',
       'Palermo Chico', 'Villa Luro', 'Villa Devoto', 'Boca',
       'Parque Avellaneda', 'San Cristobal', 'Velez Sarsfield', 'Abasto',
       'Versalles', 'Villa del Parque', 'Monte Castro', 'Retiro',
       'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Centro / Microcentro', 'Once', 'Tribunales', 'Parque Chacabuco',
       'Catalinas', 'Villa General Mitre', 'Palermo Viejo',
       'Villa Lugano', 'Villa Ortuzar', 'Villa Soldati', 'Villa Re

In [26]:
data.loc[data.currency.isna()]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,operation_category,property_type_category,place_name_check,place_with_parent_names_check
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|
20,20,sell,house,Villa Giardino,|Argentina|Córdoba|Villa Giardino|,Argentina,Córdoba,3832729.0,"-31.0529852,-64.4991278",-31.052985,...,NaN,NaN,http://www.properati.com.ar/15bqb_venta_casa_v...,"Consta de: Comedor-cocina, cuatro dormitorios ...","Casa con excelente ubicaciòn, calle asfaltada,...",https://thumbs4.properati.com/1/9Z3PZS2TK7olUo...,sell,house,Villa Giardino,|Argentina|Córdoba|Villa Giardino|
41,41,sell,apartment,Villa Carlos Paz,|Argentina|Córdoba|Villa Carlos Paz|,Argentina,Córdoba,3832791.0,"-31.4009989,-64.4780258",-31.400999,...,4.0,NaN,http://www.properati.com.ar/15bta_venta_depart...,“Arecas” es un complejo de dúplex y departamen...,Venta en pozo deptos-dúplex- Barrio Santa Rita...,https://thumbs4.properati.com/7/EV8UVQgLFQCAxr...,sell,apartment,Villa Carlos Paz,|Argentina|Córdoba|Villa Carlos Paz|
62,62,sell,house,Berazategui,|Argentina|Bs.As. G.B.A. Zona Sur|Berazategui|,Argentina,Bs.As. G.B.A. Zona Sur,3436042.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/15bvu_venta_casa_b...,Casa moderna en dos plantas.PB : Living con ho...,MODERNA CASA EN OMBUES DE HUDSON,https://thumbs4.properati.com/7/a8Qmcv66lP2cTc...,sell,house,Berazategui,|Argentina|Bs.As. G.B.A. Zona Sur|Berazategui|
67,67,sell,house,Ituzaingó,|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|,Argentina,Bs.As. G.B.A. Zona Oeste,3433359.0,"-34.6422298,-58.7031296",-34.642230,...,5.0,NaN,http://www.properati.com.ar/15bwz_venta_casa_i...,PLANTA ALTA: DORMITORIO EN SUITE - BALCÓNPLANT...,Chalet en Venta. Ambientes. Dormitorios. 220 m2,https://thumbs4.properati.com/6/cQAGKCZCYDfOgz...,sell,house,Ituzaingó,|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121210,121210,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cj8o_venta_casa_m...,Dos viviendas en block en excelente ubicación ...,Casa - Martinez,https://thumbs4.properati.com/7/3-sXSLx8KPDZzR...,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
121211,121211,sell,apartment,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cj9y_venta_depart...,Excelente departamento de categoría con vistas...,Departamento - Acassuso,https://thumbs4.properati.com/0/pvevdNsjE5Kf4r...,sell,apartment,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
121212,121212,sell,house,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436508.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cj9z_venta_casa_a...,ESPECTACULAR CASA ESTILO RACIONALISTA EN CALLE...,Casa - Acassuso,https://thumbs4.properati.com/7/DiXqpull13ebTp...,sell,house,Acassuso,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
121213,121213,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cja0_venta_casa_m...,EXCELENTE casa en Martinez. Inmejorable ubicai...,Casa - Martinez,https://thumbs4.properati.com/8/v0Il0v9Key--5Z...,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...


In [27]:
mask1 = (data['currency'].isna()) \
        & (data['price'].isna()) \
        & (data['price_aprox_local_currency'].isna()) \
        & (data['price_aprox_usd'].isna()) \
        & (data['price_usd_per_m2'].isna()) \
        & (data['price_per_m2'].isna())

mask2 = (data['price'].isna()) \
        & (data['price_aprox_local_currency'].isna()) \
        & (data['price_aprox_usd'].isna()) \
        & (data['price_usd_per_m2'].isna()) \
        & (data['price_per_m2'].isna())

#print(mask1.value_counts())
#print(mask2.value_counts())
data.loc[mask2,:].head(5)['description']

5     Casa en el perímetro del barrio 338, ubicada e...
20    Consta de: Comedor-cocina, cuatro dormitorios ...
41    “Arecas” es un complejo de dúplex y departamen...
62    Casa moderna en dos plantas.PB : Living con ho...
67    PLANTA ALTA: DORMITORIO EN SUITE - BALCÓNPLANT...
Name: description, dtype: object

In [28]:
data.iloc[67,:][{'description','title'}].description

'PLANTA ALTA: DORMITORIO EN SUITE - BALCÓNPLANTA BAJA: 2 DORMITORIOS - BAÑO - COCINA - LIVING/COMEDOR - LAVADERO - PATIO- FONDO LIBRE- ENTRADA DE AUTO - LOCAL CON BAÑOCONSULTE VALOR'

In [29]:
data.describe()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000000,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,60609.500000,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,34993.344153,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,0.000000,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,30304.750000,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,60609.500000,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,90914.250000,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,121219.000000,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [30]:
df['description'].sample(10)

NameError: name 'df' is not defined

In [ ]:
df.info()

In [ ]:
import numpy as np
col_floor = df.floor

In [ ]:
df[df[]]

In [ ]:
df.iloc[121217].geonames_id

In [ ]:
df_map_test = df[~df['lat'].isna()]
df_map_test.sample(30000)

In [ ]:
df.iloc[26637]

In [ ]:
df.info()

In [ ]:
df